In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import glob

import math
from scipy import linalg
from numpy.linalg import inv
from numpy import linalg as LA
from sklearn import linear_model, datasets
from function import *

In [2]:
img_p, corners = return_imagepoints()
obj_p = return_objpoints()

obj_p4 = homo_obj4(obj_p)
obj_p3 = homo_obj3(obj_p)
img_p = homo_img(img_p)

print("obj_p3.shape", obj_p3.shape)
print("obj_p4.shape", obj_p4.shape)
print("img_p.shape", img_p.shape)

obj_p3.shape (3, 315)
obj_p4.shape (4, 315)
img_p.shape (3, 315)


In [3]:
# img_p[:2,:].T.shape
# obj_p[:2,:].T.shape
img_p[:2,:].shape

(2, 315)

In [4]:
obj_pt3 = obj_p3.T
obj_pt4 = obj_p4.T
img_pt = img_p.T
print("obj_pt.shape", obj_pt3.shape)
print("obj_pt.shape", obj_pt4.shape)
print("img_pt.shape", img_pt.shape)

obj_pt.shape (315, 3)
obj_pt.shape (315, 4)
img_pt.shape (315, 3)


In [5]:
objpoints = [] 
imgpoints = []

imgpoints.append(corners)
objpoints.append(obj_p)

img = cv2.imread("./a.png")
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

rx = [[1,0,0],[0,math.cos(rvecs[0][0]),math.sin(rvecs[0][0])],[0,(-1)*math.sin(rvecs[0][0]),math.cos(rvecs[0][0])]]

ry = [[math.cos(rvecs[0][1]),0,math.sin(rvecs[0][1])],[0,1,0],[(-1)*math.sin(rvecs[0][1]),0,math.cos(rvecs[0][1])]]

rz = [[math.cos(rvecs[0][2]),math.sin(rvecs[0][2]),0],[(-1)*math.sin(rvecs[0][2]),math.cos(rvecs[0][2]),0],[0,0,1]]

R1 = np.matmul(rx,ry)
R = np.matmul(R1,rz)
print("rotation_matrix",R)
print("tvecs",tvecs)

m = np.ones((3,4))
m[:,:3] = R
m[:,[3]]= np.array(tvecs)

cv_homography = np.matmul(mtx,m)

pcv = cv_homography/cv_homography[-1,-1]

rotation_matrix [[ 0.9992851   0.0046721   0.0375162 ]
 [-0.0048877   0.99997205  0.0056571 ]
 [-0.03748872 -0.00583643  0.99928001]]
tvecs [array([[-9.46710699],
       [-5.90093295],
       [63.95609032]])]


In [6]:
p = projection_matrix_direct(img_pt,obj_pt3)
print(p,"\n")

p_ = projection_matrix3(img_pt,obj_pt3)
print(p_,"\n")

_p = projection_matrix4(img_pt,obj_pt4)
print(_p,"\n")

print(pcv)

[[ 2.35641141e+01 -1.13180327e-01  1.07795469e+02]
 [ 1.24120961e-01  2.34434472e+01  8.29785775e+01]
 [ 2.21177243e-17 -6.93889390e-17  1.00000000e+00]] 

[[ 2.32708135e+01 -6.48353739e-02  1.08385471e+02]
 [-1.23646156e-02  2.33727681e+01  8.37870163e+01]
 [-5.46339618e-04  1.40990039e-04  1.00000000e+00]] 

[[ 1.36930779e+02 -3.50209450e-01  5.00613492e-02  1.08329258e+02]
 [-5.06145912e-02  1.36930340e+02 -1.36930779e+02  8.37485083e+01]
 [-3.13732526e-03  8.94422047e-04  0.00000000e+00  1.00000000e+00]] 

[[ 2.29941872e+01  7.82454466e-02  6.03584809e+00  1.10910486e+02]
 [-2.42857211e-01  2.30889446e+01  3.59337234e+00  8.52468687e+01]
 [-5.86163404e-04 -9.12567791e-05  1.56244699e-02  1.00000000e+00]]


In [7]:
# a,b,c = RQ_decomposition(_p)
# print("camera_calibration_matrix",a)
# print("rotation_matrix",b)
# print("translation_matrix",c)

NOTE: The (1,2) term is zero in a camera calibration matrix, which is not zero in our case.

Manually decomposing the p-matrix by RQ decomposition.

In [8]:
# # trying to get the camera matrix in proper form
# m = _p[:,:3]
# m1 = m[:,0]
# m2 = m[:,1]
# m3 = m[:,2]

# alpha1 = LA.norm(m1)
# alpha2 = LA.norm(m2)
# x0 = LA.norm(m3)
# y0 = LA.norm(m3)
# print(alpha1,alpha2,x0,y0)
# # kappa = [[alpha1,0,x0],[0,alpha2,y0],[0,0,1]]
# # kappa
# # print(_p[0])
# # print(_p[1])
# # print(_p[2])

On manually calculating the RQ decomposition, in the k-matrix the value of all the variable comes out to be 136.9, this is because in all the columns of p-matrix, there is a dominting value (1.36) present.

In [9]:
proj_cv = img_projection(pcv, np.array([[10,20,100]]))
print(proj_cv)

_proj = img_projection(_p,np.array([[10,20,100]]))
print(_proj)

p = img_projection(_p,obj_p[:3,:])
p

[[370.28996658]
 [353.82358638]
 [  1.        ]]
[[ 1.4958097e+03]
 [-1.1019829e+04]
 [ 1.0000000e+00]]


array([[109.01327782, 246.80867434, 384.60407087],
       [ 84.27731901,  84.22638482,  84.17545063],
       [  1.00631427,   1.00315714,   1.        ]])

--------------------------------------

---------------------------------------